In [195]:
# Initial imports
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import nltk as nltk
from wordcloud import WordCloud
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
load_dotenv()
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta
import hvplot.pandas

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\bfode\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [196]:
# Read your api key environment variable
# YOUR CODE HERE!
api_key = os.getenv("news_api")

In [197]:
# Create a newsapi client
# YOUR CODE HERE!
newsapi = NewsApiClient(api_key=api_key)

In [198]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["news_api"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [199]:
# Get last 30days' worth of historical data for GS 
#Wells Fargo Co. (WFC), Goldman Sachs Group Inc. (GS), and Morgan Stanley (MS).

# Set the ticker
ticker = "GS"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.head()

GS                                    
                             open      high      low   close   volume
2021-06-08 00:00:00-04:00  385.79  386.4799  382.220  384.80  1946972
2021-06-09 00:00:00-04:00  383.44  384.2700  378.880  382.78  1817307
2021-06-10 00:00:00-04:00  389.08  389.6400  372.345  373.75  3302197
2021-06-11 00:00:00-04:00  375.47  378.7500  375.110  378.23  1692723
2021-06-14 00:00:00-04:00  377.43  378.7500  370.700  373.07  2023651

In [200]:
#Saving Raw data to Raw Data Folder
df.to_csv('../../data/Rawdata/gs_stock_15min.csv')

In [201]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2021-06-08,384.80
2021-06-09,382.78
2021-06-10,373.75
2021-06-11,378.23
2021-06-14,373.07


In [202]:
df_returns = df.pct_change().dropna()
df_returns.head()

,close
2021-06-09,-0.005249
2021-06-10,-0.023591
2021-06-11,0.011987
2021-06-14,-0.013642
2021-06-15,-0.003940


In [203]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [204]:
# Fetching headline
goldman_headlines, dates = get_headlines("Goldman Sachs Group Inc.")

Fetching news about 'Goldman Sachs Group Inc.'
******************************
retrieving news from: 2021-07-08 00:00:00
retrieving news from: 2021-07-07 00:00:00
retrieving news from: 2021-07-06 00:00:00
retrieving news from: 2021-07-05 00:00:00
retrieving news from: 2021-07-04 00:00:00
retrieving news from: 2021-07-03 00:00:00
retrieving news from: 2021-07-02 00:00:00
retrieving news from: 2021-07-01 00:00:00
retrieving news from: 2021-06-30 00:00:00
retrieving news from: 2021-06-29 00:00:00
retrieving news from: 2021-06-28 00:00:00
retrieving news from: 2021-06-27 00:00:00
retrieving news from: 2021-06-26 00:00:00
retrieving news from: 2021-06-25 00:00:00
retrieving news from: 2021-06-24 00:00:00
retrieving news from: 2021-06-23 00:00:00
retrieving news from: 2021-06-22 00:00:00
retrieving news from: 2021-06-21 00:00:00
retrieving news from: 2021-06-20 00:00:00
retrieving news from: 2021-06-19 00:00:00
retrieving news from: 2021-06-18 00:00:00
retrieving news from: 2021-06-17 00:00:0

In [205]:
df = pd.DataFrame(goldman_headlines)
goldman_headlines#.to_csv('../../data/Rawdata/goldman_headlines.csv')


[['Chinese Startup Bianlifeng Files Confidentially\xa0for U.S. IPO',
  "There's Been A Clear Change In Leadership (Technically Speaking For 7/7)",
  'China Hints at Reserve Ratio Cut to Help Bolster Economy',
  'The Great Reflation Trade Is Buckling Around the World',
  'Visa says spending on crypto-linked cards topped $1 bn in first half this year',
  'US IPO market a danger zone for Chinese firms after Beijing crackdown',
  'India Supercharged Its Economy 30 Years Ago. Covid Unraveled It In Months',
  'Covid undoes gains India made after supercharging economy 30 years ago',
  'India supercharged its economy 30 years ago. Covid unraveled it in months',
  'India supercharged its economy 30 years ago. COVID-19 unraveled it in months.'],
 ['Aéropostale, Sports Illustrated owner files for U.S. IPO - Reuters',
  'Google-backed Planet Labs to go public in $2.8 billion SPAC deal - Reuters',
  'Visa says spending on crypto-linked cards topped $1 bln in first half this year - Reuters',
  'Goog

In [206]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [231]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [232]:
# Get averages of each topics sentiment
goldman_avg = headline_sentiment_summarizer_avg(goldman_headlines)


In [233]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "goldman_avg": goldman_avg
    }
)

In [234]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

In [235]:
# Merge with goldman returns
topic_sentiments = df_returns.join(topic_sentiments).dropna(how="any")

# Display data
display(topic_sentiments)

,close,goldman_avg
2021-06-09,-0.005249,0.036050
2021-06-10,-0.023591,0.135205
2021-06-11,0.011987,0.081830
2021-06-14,-0.013642,0.014035
2021-06-15,-0.003940,0.159560
2021-06-16,-0.001426,0.084235
2021-06-17,-0.025790,0.102160
2021-06-18,-0.035159,0.169720
2021-06-21,0.025488,0.342390
2021-06-22,-0.000419,0.103825


In [212]:
topic_sentiments.corr().style.background_gradient()

,close,goldman_avg
close,1.000000,0.215188
goldman_avg,0.215188,1.000000


# Build and Train the LSTM RNN

In [213]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [214]:
# Predict Closing Prices using a 10 day window of previous fng values
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 1 is the 'Goldman Average' column
# Column index 0 is the `Close` column
feature_column = 1
target_column = 0
X, y = window_data(topic_sentiments, window_size, feature_column, target_column)

In [215]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.4 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

In [216]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [217]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [218]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [219]:
#Build the LSTM model. 
model = Sequential()

number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [220]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [221]:
# Summarize the model
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 10, 30)            3840      
_________________________________________________________________
dropout_24 (Dropout)         (None, 10, 30)            0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 10, 30)            7320      
_________________________________________________________________
dropout_25 (Dropout)         (None, 10, 30)            0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dropout_26 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                

In [222]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
# YOUR CODE HERE!
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)


Epoch 1/10
3/3 [==============================] - 4s 6ms/step - loss: 0.5120
Epoch 2/10
3/3 [==============================] - 0s 6ms/step - loss: 0.4511
Epoch 3/10
3/3 [==============================] - 0s 6ms/step - loss: 0.4073
Epoch 4/10
3/3 [==============================] - 0s 7ms/step - loss: 0.2981
Epoch 5/10
3/3 [==============================] - 0s 7ms/step - loss: 0.2421
Epoch 6/10
3/3 [==============================] - 0s 6ms/step - loss: 0.1774
Epoch 7/10
3/3 [==============================] - 0s 7ms/step - loss: 0.1146
Epoch 8/10
3/3 [==============================] - 0s 9ms/step - loss: 0.1169
Epoch 9/10
3/3 [==============================] - 0s 7ms/step - loss: 0.0976
Epoch 10/10
3/3 [==============================] - 0s 7ms/step - loss: 0.1500


In [223]:
# Evaluate the model
model.evaluate(X_test, y_test)

1/1 [==============================] - 1s 1s/step - loss: 0.4441


0.44405412673950195

In [224]:
# Make some predictions
predicted = model.predict(X_test)

In [225]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [226]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
24,-0.000203,0.017772
25,0.010920,0.020056
26,0.018316,0.019659
27,-0.011820,0.020236
28,-0.002134,0.020267


In [238]:
# Plot the real vs predicted values as a line chart
# YOUR CODE HERE!
stocks.hvplot.line(xlabel="Date",ylabel="Ruturns", width=850, title='USING NLP SCENTIMENT SCORE TO PREDICT GOLDMAN SACH RETURNS' )

:NdOverlay   [Variable]
   :Curve   [index]   (value)